In [1]:
# RGB: https://www.rapidtables.com/web/color/RGB_Color.html
import json
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
import glob
np.random.seed(251)

In [2]:
project_path = r'D:\UCBerkeley\CourseWork\202001\W251\Homework\Project'
dir_video = r'openpose_output\video_NEW'
dir_json = r'openpose_output\json_NEW'
dir_optical_flow = r'openpose_output\manual_optical_flow_output_hand4'
dir_transfer = r'openpose_output\image_transfer_hand4'

class_list = ['AGAIN', 'ALL', 'AWKWARD', 'BASEBALL', 'BEHAVIOR', 'CAN', 'CHAT', 'CHEAP', 
              'CHEAT', 'CHURCH', 'COAT', 'CONFLICT', 'COURT', 'DEPOSIT', 'DEPRESS', 
              'DOCTOR', 'DRESS', 'ENOUGH', 'NEG']

body_feature_set = [3, 4, 6, 7]
hand_feature_set = list(range(21))
hand_feature_set1 = [0, 17, 13, 9, 5, 1]
hand_feature_set2 = [0, 18, 14, 10, 6, 2]
hand_feature_set3 = [0, 19, 15, 11, 7, 3]
hand_feature_set4 = [0, 20, 16, 12, 8, 4]

# Create some random colors
color = np.random.randint(0,255,(300,3))

In [3]:
# Remove all the files in dir_optical_flow
files = glob.glob(os.path.join(project_path, dir_optical_flow, '*'))
for f in files:
    os.remove(f)
# Remove all the files in the dir_transfer folder
for i in range(len(class_list)):
    files = glob.glob(os.path.join(project_path, dir_transfer, class_list[i], '*'))
    for f in files:
        os.remove(f)


In [4]:
mov_file_lst = [f for f in os.listdir((os.path.join(project_path, dir_video)))]
for mov_file in mov_file_lst:
    print(mov_file)
    json_files_lst = [f for f in os.listdir(os.path.join(project_path, dir_json)) if os.path.splitext(mov_file)[0] in f]
    if len(json_files_lst) > 0:
        print('Found the json files')
    feature_body_dict = dict()
    feature_hand_left_dict = dict()
    feature_hand_right_dict = dict()
    
    for json_f in json_files_lst:
        with open(os.path.join(project_path, dir_json, json_f)) as ff:
            json_code = json.load(ff)
        
        # This assume there is only one person
        body_raw_lst = json_code['people'][0]['pose_keypoints_2d']
        left_hand_raw_lst = json_code['people'][0]['hand_left_keypoints_2d']
        right_hand_raw_lst = json_code['people'][0]['hand_right_keypoints_2d']
        
        for i in body_feature_set:
            temp = feature_body_dict.get(i, [])
            temp.append(body_raw_lst[3*i:3*(i+1)])
            feature_body_dict[i] = temp
            
        for i in hand_feature_set4:
            temp = feature_hand_left_dict.get(i, [])
            temp.append(left_hand_raw_lst[3*i:3*(i+1)])
            feature_hand_left_dict[i] = temp
            
        for i in hand_feature_set4:
            temp = feature_hand_right_dict.get(i, [])
            temp.append(right_hand_raw_lst[3*i:3*(i+1)])
            feature_hand_right_dict[i] = temp
    
    cap = cv2.VideoCapture(os.path.join(project_path, dir_video, mov_file))
    ret, first_frame = cap.read()
    mask = np.zeros_like(first_frame)
    for (k, v) in feature_body_dict.items():
        x_0 = int(v[0][0])
        y_0 = int(v[0][1])
        for points in v[1:]:
            x_1 = int(points[0])
            y_1 = int(points[1])
            if x_0 == 0 and y_0 == 0:
                x_0 = x_1
                y_0 = y_1
            if x_1 != 0 and y_1 != 0:
                mask = cv2.line(mask, (x_0, y_0), (x_1, y_1), color[k].tolist(), 5)
                x_0 = x_1
                y_0 = y_1
                
    for (k, v) in feature_hand_left_dict.items():
        x_0 = int(v[0][0])
        y_0 = int(v[0][1])
        for points in v[1:]:
            x_1 = int(points[0])
            y_1 = int(points[1])
            if x_0 == 0 and y_0 == 0:
                x_0 = x_1
                y_0 = y_1
            if x_1 != 0 and y_1 != 0:
                mask = cv2.line(mask, (x_0, y_0), (x_1, y_1), color[k+50].tolist(), 1)
                x_0 = x_1
                y_0 = y_1
                
    for (k, v) in feature_hand_right_dict.items():
        x_0 = int(v[0][0])
        y_0 = int(v[0][1])
        for points in v[1:]:
            x_1 = int(points[0])
            y_1 = int(points[1])
            if x_0 == 0 and y_0 == 0:
                x_0 = x_1
                y_0 = y_1
            if x_1 != 0 and y_1 != 0:
                mask = cv2.line(mask, (x_0, y_0), (x_1, y_1), color[k+100].tolist(), 1)
                x_0 = x_1
                y_0 = y_1
    
    temp_segments = len(mov_file.split('_'))
    word = mov_file.split('_')[temp_segments-4]
    counter = mov_file.split('_')[temp_segments-2]
    train_test = mov_file.split('_')[temp_segments-1].split('.')[0]
    save_file = 'mask_' + word + '_' + counter + '_' + train_test
    print('Save as', save_file)
    plt.imsave(os.path.join(project_path, dir_optical_flow, save_file + '.png'), mask)
    if 'train' in save_file:
        plt.imsave(os.path.join(project_path, dir_transfer, word, save_file + '.png'), mask)
    cap.release()
    cv2.destroyAllWindows()


ASL_2006_10_10_scene2-camera1_AGAIN_v0_c9_train.mov
Found the json files
Save as mask_AGAIN_c9_train
ASL_2006_10_10_scene2-camera1_ALL_v0_c4_train.mov
Found the json files
Save as mask_ALL_c4_train
ASL_2006_10_10_scene3-camera1_AWKWARD_v0_c4_test.mov
Found the json files
Save as mask_AWKWARD_c4_test
ASL_2006_10_10_scene3-camera1_BASEBALL_v1_c1_train.mov
Found the json files
Save as mask_BASEBALL_c1_train
ASL_2006_10_10_scene4-camera1_CAN_v0_c6_test.mov
Found the json files
Save as mask_CAN_c6_test
ASL_2006_10_10_scene4-camera1_CHAT_v2_c8_train.mov
Found the json files
Save as mask_CHAT_c8_train
ASL_2006_10_10_scene5-camera1_CHEAP_v0_c4_train.mov
Found the json files
Save as mask_CHEAP_c4_train
ASL_2006_10_10_scene5-camera1_CHURCH_v1_c4_train.mov
Found the json files
Save as mask_CHURCH_c4_train
ASL_2006_10_10_scene5-camera1_COAT_v0_c4_train.mov
Found the json files
Save as mask_COAT_c4_train
ASL_2006_10_10_scene6-camera1_CONFLICT_v0_c7_train.mov
Found the json files
Save as mask_CONFLI